# LangServe

- LangServe는 LangChain에서 제공하는 **서비스 배포 프레임워크**임.

- LangChain 애플리케이션을 **REST API로 변환**하여 서비스화함.

- **Runnable 객체**를 API로 배포하고 **스트리밍 응답**을 지원함.

- **웹 기반 Playgroud**(API 테스트용)와 **클라이언트 SDK**를 제공함.


---

## 1. LangServe 설치하기

- LangServe 설치(pip 사용):

    ```bash
    pip install "langserve[all]"
    ```

- LangServe 설치(uv 사용):

    ```bash
    uv add "langserve[all]"   
    ```

- langchain-cli 설치:

    ```bash
    uv add langchain-cli
    ```

`(1) LangServe 설치 확인`

In [1]:
import langserve
langserve.__version__

'0.3.1'

`(2) 환경 변수 로드`

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---
## 2. 간단한 LangServe 서버 만들기  

`(1) 서버 파일 생성`
- app 폴더에 별도 파일을 생성하여 LangServe 서버를 구현함. 
- server.py라는 이름으로 생성함.

    ```python
    # app/server.py
    from fastapi import FastAPI
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv

    # 환경변수 로드
    load_dotenv()

    from langserve import add_routes

    # FastAPI 서버를 설정
    app = FastAPI(
        title="LangChain Server",
        version="1.0",
        description="Spin up a simple api server using Langchain's Runnable interfaces",
    )

    # 라우팅 설정
    add_routes(
        app,
        ChatOpenAI(model="gpt-4.1-mini"),
        path="/openai",   # OpenAI 모델에 대한 경로
    )

    if __name__ == "__main__":
        import uvicorn

        uvicorn.run(app, host="localhost", port=8000)
    ```

`(2) 서버 실행하기`


```bash
uv run python app/server.py
```

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0001.png)

`(3) Playground에서 테스트`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0002.png)

`(4) LangSmith에서 트레이싱하기`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0003.png)

---
## 3. LangServe 클라이언트 사용  

LangServe는 다음과 같은 API 엔드포인트를 자동으로 생성합니다:

- `/openai/invoke`: 동기 호출
- `/openai/stream`: 스트리밍 호출
- `/openai/batch`: 배치 처리
- `/openai/playground`: 웹 기반 UI 플레이그라운드

`(1) API 호출하기`

In [3]:
# - `/openai/invoke`: 동기 호출

import requests

url = "http://localhost:8000/openai/invoke"
headers = {
    "Content-Type": "application/json",
}

data = {
    "input": "파이썬이란 무엇인가요?"
}

response = requests.post(url, headers=headers, json=data)
print(response.status_code)
print(response.json())

200
{'output': {'content': '파이썬(Python)은 배우기 쉽고 강력한 고급 프로그래밍 언어입니다. 1991년 귀도 반 로섬(Guido van Rossum)이 개발하였으며, 명확하고 간결한 문법을 가지고 있어 초보자부터 전문가까지 폭넓게 사용됩니다.\n\n파이썬의 주요 특징은 다음과 같습니다:\n\n1. **읽기 쉽고 쓰기 쉬운 문법**: 코드가 직관적이고 가독성이 뛰어나서 유지보수와 협업에 유리합니다.\n2. **다양한 용도**: 웹 개발, 데이터 분석, 인공지능, 과학 계산, 자동화, 게임 개발 등 여러 분야에서 활용됩니다.\n3. **방대한 표준 라이브러리**: 다양한 기능을 수행하는 모듈과 패키지가 기본 제공되어 개발 생산성을 높여줍니다.\n4. **인터프리터 언어**: 코드를 한 줄씩 실행하기 때문에 빠른 테스트와 디버깅이 가능합니다.\n5. **크로스 플랫폼**: 윈도우, 맥, 리눅스 등 여러 운영체제에서 실행할 수 있습니다.\n6. **오픈 소스**: 누구나 무료로 사용하고 수정할 수 있습니다.\n\n예를 들어, 간단한 "Hello, World!" 출력 코드는 다음과 같습니다:\n\n```python\nprint("Hello, World!")\n```\n\n파이썬은 그 사용의 용이성과 강력한 기능 덕분에 전 세계적으로 매우 인기 있는 프로그래밍 언어 중 하나입니다.', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 338, 'prompt_tokens': 17, 'total_tokens': 355, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio

`(2) Python에서 클라이언트 SDK 사용하기`

In [4]:
from langserve.client import RemoteRunnable

# 원격 실행 가능 객체를 생성
chain = RemoteRunnable("http://localhost:8000/openai")

# API 호출
response = chain.invoke("파이썬이란 무엇인가요?")
print(response)

content='파이썬(Python)은 고급 프로그래밍 언어 중 하나로, 1991년 네덜란드의 프로그래머 귀도 반 로섬(Guido van Rossum)이 개발했습니다. 파이썬은 문법이 간결하고 읽기 쉬워 초보자부터 전문가까지 널리 사용되며, 다양한 분야에서 활용되고 있습니다.\n\n주요 특징은 다음과 같습니다:\n\n1. **쉬운 문법**: 들여쓰기를 통해 코드 블록을 구분하므로 가독성이 뛰어납니다.\n2. **인터프리터 언어**: 한 줄씩 코드를 실행하기 때문에 개발과 디버깅이 용이합니다.\n3. **풍부한 라이브러리**: 데이터 분석, 인공지능, 웹 개발 등 다양한 분야에 활용할 수 있는 방대한 표준 및 외부 라이브러리를 제공합니다.\n4. **다양한 용도**: 웹 개발, 데이터 과학, 인공지능, 자동화, 스크립트 작성 등 여러 분야에서 사용됩니다.\n5. **크로스 플랫폼**: Windows, macOS, Linux 등 다양한 운영체제에서 실행 가능합니다.\n\n예를 들어, 파이썬으로 간단한 인사말을 출력하는 코드는 다음과 같습니다:\n\n```python\nprint("안녕하세요, 파이썬!")\n```\n\n이처럼 파이썬은 배우기 쉽고 강력한 기능을 갖춰 많은 프로그래머들이 선호하는 언어입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 321, 'prompt_tokens': 17, 'total_tokens': 338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-

In [5]:
response

AIMessage(content='파이썬(Python)은 고급 프로그래밍 언어 중 하나로, 1991년 네덜란드의 프로그래머 귀도 반 로섬(Guido van Rossum)이 개발했습니다. 파이썬은 문법이 간결하고 읽기 쉬워 초보자부터 전문가까지 널리 사용되며, 다양한 분야에서 활용되고 있습니다.\n\n주요 특징은 다음과 같습니다:\n\n1. **쉬운 문법**: 들여쓰기를 통해 코드 블록을 구분하므로 가독성이 뛰어납니다.\n2. **인터프리터 언어**: 한 줄씩 코드를 실행하기 때문에 개발과 디버깅이 용이합니다.\n3. **풍부한 라이브러리**: 데이터 분석, 인공지능, 웹 개발 등 다양한 분야에 활용할 수 있는 방대한 표준 및 외부 라이브러리를 제공합니다.\n4. **다양한 용도**: 웹 개발, 데이터 과학, 인공지능, 자동화, 스크립트 작성 등 여러 분야에서 사용됩니다.\n5. **크로스 플랫폼**: Windows, macOS, Linux 등 다양한 운영체제에서 실행 가능합니다.\n\n예를 들어, 파이썬으로 간단한 인사말을 출력하는 코드는 다음과 같습니다:\n\n```python\nprint("안녕하세요, 파이썬!")\n```\n\n이처럼 파이썬은 배우기 쉽고 강력한 기능을 갖춰 많은 프로그래머들이 선호하는 언어입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 321, 'prompt_tokens': 17, 'total_tokens': 338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'g